In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
tables = ['Employee', 'Department']
for table in tables:
    display_table(table)

+----+-------+--------+--------------+
| id |  name | salary | departmentId |
+----+-------+--------+--------------+
| 1  |  Joe  | 85000  |      1       |
| 2  | Henry | 80000  |      2       |
| 3  |  Sam  | 60000  |      2       |
| 4  |  Max  | 90000  |      1       |
| 5  | Janet | 69000  |      1       |
| 6  | Randy | 85000  |      1       |
| 7  |  Will | 70000  |      1       |
+----+-------+--------+--------------+
+----+-------+
| id |  name |
+----+-------+
| 1  |   IT  |
| 2  | Sales |
+----+-------+


#### *A company's executives are interested in seeing who earns the most money in each of the company's departments. A high earner in a department is an employee who has a salary in the top three unique salaries for that department.*

### Write an SQL query to find the employees who are high earners in each of the departments.

```
+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Max      | 90000  |
| IT         | Joe      | 85000  |
| IT         | Randy    | 85000  |
| IT         | Will     | 70000  |
| Sales      | Henry    | 80000  |
| Sales      | Sam      | 60000  |
+------------+----------+--------+
Explanation: 
In the IT department:
- Max earns the highest unique salary
- Both Randy and Joe earn the second-highest unique salary
- Will earns the third-highest unique salary

In the Sales department:
- Henry earns the highest salary
- Sam earns the second-highest salary
- There is no third-highest salary as there are only two employees
```

In [5]:
%%sql 

SELECT 
    d.name AS Department,
    e.name AS Employee,
    e.salary
FROM Employee e
JOIN Department d ON e.departmentId = d.id 

Department,Employee,salary
IT,Joe,85000
Sales,Henry,80000
Sales,Sam,60000
IT,Max,90000
IT,Janet,69000
IT,Randy,85000
IT,Will,70000


In [10]:
%%sql 

SELECT 
    salary 
FROM Employee e
JOIN Department d ON e.departmentId = d.id 
WHERE d.name = 'IT'

salary
85000
90000
69000
85000
70000


In [30]:
%%sql 

SELECT 
    DISTINCT salary 
FROM Employee e
JOIN Department d ON e.departmentId = d.id 
WHERE d.name = 'IT'
ORDER BY salary DESC
LIMIT 3

salary
90000
85000
70000


In [31]:
%%sql 

SELECT 
    DISTINCT salary 
FROM Employee e
JOIN Department d ON e.departmentId = d.id 
WHERE d.name = 'Sales'
ORDER BY salary DESC
LIMIT 3

salary
80000
60000


In [55]:
%%sql 

SELECT 
    d.name AS Department,
    e.name AS Employee,
    e.salary
FROM Employee e
JOIN Department d ON e.departmentId = d.id 
WHERE d.name = 'IT' AND e.salary IN (
    SELECT 
        DISTINCT salary 
    FROM Employee e
    JOIN Department d ON e.departmentId = d.id 
    WHERE d.name = 'IT'
    ORDER BY salary DESC
)
ORDER BY e.salary DESC

Department,Employee,salary
IT,Max,90000
IT,Randy,85000
IT,Joe,85000
IT,Will,70000
IT,Janet,69000


In [56]:
%%sql 

SELECT 
    d.name AS Department,
    e.name AS Employee,
    e.salary
FROM Employee e
JOIN Department d ON e.departmentId = d.id 
WHERE d.name = 'Sales' AND e.salary IN (
    SELECT 
        DISTINCT salary 
    FROM Employee e
    JOIN Department d ON e.departmentId = d.id 
    WHERE d.name = 'Sales'
    ORDER BY salary DESC
)
ORDER BY e.salary DESC

Department,Employee,salary
Sales,Henry,80000
Sales,Sam,60000


In [59]:
%%sql 

(SELECT 
    d.name AS Department,
    e.name AS Employee,
    e.salary
FROM Employee e
JOIN Department d ON e.departmentId = d.id 
WHERE d.name = 'IT' AND e.salary IN (
    SELECT 
        DISTINCT salary 
    FROM Employee e
    JOIN Department d ON e.departmentId = d.id 
    WHERE d.name = 'IT'
    ORDER BY salary DESC)
ORDER BY e.salary DESC)

UNION

(SELECT 
    d.name AS Department,
    e.name AS Employee,
    e.salary
FROM Employee e
JOIN Department d ON e.departmentId = d.id 
WHERE d.name = 'Sales' AND e.salary IN (
    SELECT 
        DISTINCT salary 
    FROM Employee e
    JOIN Department d ON e.departmentId = d.id 
    WHERE d.name = 'Sales'
    ORDER BY salary DESC)
ORDER BY e.salary DESC)

Department,Employee,salary
IT,Randy,85000
IT,Joe,85000
IT,Max,90000
IT,Janet,69000
IT,Will,70000
Sales,Henry,80000
Sales,Sam,60000
